# Anthropic Claudeを使用した小規模ファイルのテキスト要約

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

## 概要

この例では、少量のデータ (文字列データ) を直接 Amazon Bedrock API (Anthropic Claude モデルを使用) に取り込み、それぞれのテキストを要約するよう指示を出します。

### アーキテクチャ

![](./images/41-text-simple-1.png)

アーキテクチャ:

1. 小さなテキスト (または小さなファイル) が読み込まれます
1. 基盤モデルが入力データを処理します
1. モデルは、取り込まれたテキストの要約を含む応答を返します

### ユースケース

このアプローチは、通話記録、会議記録、書籍、記事、ブログ記事などの関連コンテンツを要約するために使用できます。

### 課題

このアプローチは、入力テキストまたはファイルがモデルのコンテキスト長に収まる場合にのみ使用できます。後ほど実施するノートブック Lab2b_jp.ipynb では、ユーザーがトークン制限を超える大規模なドキュメントを持っている場合の課題に対処するアプローチを探究します。

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Boto3 を使って短いテキストの要約
 
このノートブックでは Amazon Bedrock への API リクエストの詳細を学ぶために、Boto3 をラップしてよりシンプルな API を提供する LangChain ではなく、直接 Boto3 操作してリクエストを送信する方法を紹介します。

### Request Syntax of InvokeModel in Boto3

基盤モデルにリクエストを送信するために、`InvokeModel` API を使用します。Anthropic Claude にテキストを送信する API リクエストの例を以下に示します。  `textGenerationConfig` の推論パラメータは、使用するモデルに依存します。Anthropic Claude の推論パラメータは以下の通りです:

- **temperature** - 生成のランダム性の度合いを調整します。`Temperature` が低いほどランダムな生成が少なくなります。
- **top_p** - 1 未満の場合、生成に使用する確率の合計が top_p 以上となるように、最も確率の高いトークンの最小セットのみを保持します。
- **top_k** - Top K はモデルが単語を選択する範囲のカットオフを定義します。例えば、K=50 の場合、モデルは、あるシーケンスで、次に来る可能性の高い 50 の単語から次の単語を選択します。これにより、異常な単語がシーケンスの次に選択される確率が低下します。
- **max_tokens_to_sample** - 生成する最大トークン数です。応答が最大の望ましい長さまで埋められるとは限りません。
- **stop_sequences** - API がそれ以上トークンを生成しないシーケンスです。返されたテキストには停止シーケンスは含まれません。

```python
response = bedrock.invoke_model(
    body={
        "prompt":"this is where you place your input text",
        "max_tokens_to_sample":4096,
        "temperature":0.5,
        "top_k":250,
        "top_p":0.5,
        "stop_sequences":[]
    },
    modelId="anthropic.claude-v2:1", # :1 追加
    accept=accept, 
    contentType=contentType
)
```

### 要約するテキストを含むプロンプトの記述

このノートブックでは、トークン数が基盤モデルの最大トークン数以下の短いテキストを使用できます。短いテキストの例として、Amazon Bedrock の発表に関する [AWSブログ](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/)  の記事のある段落を使用します。

プロンプトは「以下のテキストの要約を提供してください。」という指示で始まります。

In [ ]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

## プロンプトと推論パラメータを設定しリクエスト本文を作成

`invoke_model` のリクエスト構文に従い、上記のプロンプトと推論パラメーターを使用してリクエスト本文を作成します。

In [ ]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## Boto3 を使って基盤モデルを実行

ここでは、リクエストパラメーター `modelId`、`accept`、`contentType` を指定して、Amazon Bedrock に API リクエストを送信します。 プロンプトに従って、Amazon Bedrock の基盤モデルがテキストを要約します。

In [ ]:
modelId = 'anthropic.claude-v2:1' # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

上記では、Bedrock サービスは指定されたプロンプトに対して 1 つの出力として全体の要約を生成しています。出力に多数のトークンが含まれる場合、作成に時間がかかる可能性があります。

以下では、ユーザーがモデルの出力を即時に利用できるように、Bedrock の出力をストリーミングするオプションを確認します。Bedrock は、チャンクの形式で出力をストリーミングする invoke_model_with_response_stream API をサポートしています。

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

Bedrock は全体の出力を生成する代わりに、モデルからより小さなチャンクを送信します。これにより、即時利用可能な方法で応答を表示できます。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## まとめ

これで、Amazon Bedrock API のシンプルな呼び出し方法を提供する `Boto3` SDK を使用する方法を確認しました。  
この API を使用して、Amazon Bedrock に関する AWS ニュースの要約を 2 つの異なる方法で生成するユースケースを確認しました。

### ポイント

- このノートブックを使用して、Amazon Bedrock で利用できる Amazon Titan や AI21 Labs Jurassic など、異なるモデルで実験してみてください。
- プロンプトを特定のユースケースに変更し、異なるモデルの出力を評価してください。
- トークンの長さを変更して、サービスのレイテンシと反応性を理解してください。
- より良い出力を取得するために、さまざまなプロンプトエンジニアリングの原則を適用してください。

## Thank You